In [1]:
import os
os.environ['TRANSFORMERS_VERBOSITY'] = 'error'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

!pip install transformers datasets torch torchvision pillow -q

from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

print("🔄 Loading model...")

# Load model 
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

print("✅ Model loaded successfully!")

# Function to generate caption
def predict_caption(image_path):
    image = Image.open(image_path)
    inputs = processor(image, return_tensors="pt")
    out = model.generate(**inputs)
    return processor.decode(out[0], skip_special_tokens=True)

# Test with your image
caption = predict_caption("/kaggle/input/flickr8k/Images/1001773457_577c3a7d70.jpg")
print("📝 Caption:", caption)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.2 MB/s eta 0:00:0000:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 3.2 MB/s eta 0:00:00:00:0100:01m
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you

2025-11-20 13:00:26.689845: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763643627.042276      97 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763643627.173148      97 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

✅ Model loaded successfully!
📝 Caption: two dogs playing on the road


In [2]:
import os
os.environ['TRANSFORMERS_VERBOSITY'] = 'error'
os.environ['GRADIO_ANALYTICS_ENABLED'] = 'False'

import logging
logging.getLogger().setLevel(logging.ERROR)

!pip install gradio transformers torch pillow -q > /dev/null 2>&1

import gradio as gr
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
from io import BytesIO

print("🔄 Setting up AI Caption Generator...")

# Load model quietly
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

print("✅ AI Caption Generator is ready!")

def generate_caption(image):
    """Generate caption for uploaded image"""
    try:
        if image.mode != "RGB":
            image = image.convert("RGB")
        
        inputs = processor(image, return_tensors="pt")
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=50,
                num_beams=5,
                early_stopping=True
            )
        
        caption = processor.decode(outputs[0], skip_special_tokens=True)
        return caption
    
    except Exception as e:
        return "Sorry, I couldn't generate a caption for this image."

def generate_caption_from_url(url):
    """Generate caption from image URL"""
    try:
        response = requests.get(url, timeout=10)
        image = Image.open(BytesIO(response.content))
        return generate_caption(image)
    except:
        return "Please check your internet connection or try a different image URL."

# Create clean Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), title="Image Caption Generator") as demo:
    
    gr.Markdown(
        """
        <div style="text-align: center;">
        <h1>🖼 Image Caption Generator</h1>
        <p>Upload any image and get an automatic caption instantly!</p>
        </div>
        """
    )
    
    with gr.Tab("📸 Upload Image"):
        with gr.Row():
            with gr.Column():
                gr.Markdown("### Step 1: Upload your image")
                image_input = gr.Image(
                    label="",
                    type="pil",
                    height=300,
                    show_label=False
                )
                upload_btn = gr.Button("✨ Generate Caption", variant="primary", size="lg")
            
            with gr.Column():
                gr.Markdown("### Step 2: Your generated caption")
                caption_output = gr.Textbox(
                    label="",
                    placeholder="Your image caption will appear here...",
                    lines=4,
                    show_label=False
                )
        
        gr.Markdown("Try these example images:")
        gr.Examples(
            examples=[
                ["https://images.unsplash.com/photo-1541963463532-d68292c34b19?w=400"],
                ["https://images.unsplash.com/photo-1507003211169-0a1dd7228f2d?w=400"],
                ["https://images.unsplash.com/photo-1575936123452-b67c3203c357?w=400"]
            ],
            inputs=image_input,
            outputs=caption_output,
            fn=generate_caption,
            cache_examples=True,
            label=""
        )
    
    with gr.Tab("🔗 From Web URL"):
        with gr.Row():
            with gr.Column():
                gr.Markdown("### Paste image URL from web")
                url_input = gr.Textbox(
                    label="",
                    placeholder="Paste image link here...",
                    lines=2,
                    show_label=False
                )
                url_btn = gr.Button("🌐 Generate Caption", variant="primary")
            
            with gr.Column():
                url_output = gr.Textbox(
                    label="",
                    placeholder="Caption from web image...",
                    lines=4,
                    show_label=False
                )
    
    # Footer
    gr.Markdown(
        """
        ---
        <div style="text-align: center; color: #666;">
        <p>💡 <strong>Tips for better captions:</strong> Use clear, well-lit photos with recognizable objects and scenes.</p>
        <p>Perfect for: Nature photos • People • Animals • Objects • Everyday scenes</p>
        </div>
        """
    )
    
    # Event handlers
    upload_btn.click(
        fn=generate_caption,
        inputs=image_input,
        outputs=caption_output
    )
    
    url_btn.click(
        fn=generate_caption_from_url,
        inputs=url_input,
        outputs=url_output
    )

# Launch quietly
print("🚀 Starting Image Caption Generator...")
demo.launch(share=True, quiet=True, show_error=True)

🔄 Setting up AI Caption Generator...
✅ AI Caption Generator is ready!
🚀 Starting Image Caption Generator...
Caching examples at: '/kaggle/working/.gradio/cached_examples/13'
* Running on public URL: https://ddee70e62f5b436f34.gradio.live
